In [1]:
# imports
import requests
import pandas as pd

In [2]:
df = pd.read_csv('part1_station_data.csv')

In [3]:
import requests
import pandas as pd

headers = {
    "Accept": "application/json",
    "Authorization": "fsq36xroNyNlJQOc5/NZnRcJEi6Fwt7oBls6wrIyzKhZGAk="
}

foursquare_data = []
for station in df.itertuples():
    params = {
        'll': f'{station.latitude},{station.longitude}',
        'radius': 1000, 
        'categories': '13065,13003',  # Dining and Drinking > Restaurant, Dining and Drinking > Bar
        'limit': 50  
    }
    response = requests.get('https://api.foursquare.com/v3/places/search', headers=headers, params=params)

    if response.status_code != 200:
        print(f"Request failed with status code:{response.status_code}")
        continue

    data = response.json()

    for venue in data['results']:
        venue_info = {
            'name': venue['name'],
            'latitude': venue['geocodes']['main']['latitude'],
            'longitude': venue['geocodes']['main']['longitude'],
            'category': venue['categories'][0]['name'] if venue['categories'] else 'Unknown'
        }
        foursquare_data.append(venue_info)

In [4]:
foursquare_df = pd.DataFrame(foursquare_data)
foursquare_df

,name,latitude,longitude,category
0,Essers Gasthaus,50.956494,6.927119,Austrian Restaurant
1,Café Franck,50.956355,6.922496,Café
2,Pane e Cioccolata,50.954232,6.924428,Pizzeria
3,Restaurant Carls,50.955888,6.922870,German Restaurant
4,Vasco da Gama,50.955488,6.932871,Portuguese Restaurant
...,...,...,...,...
4101,Kudret Kebap House,50.948155,6.921100,Doner Restaurant
4102,Ihrefelder Eck,50.956422,6.913144,Bar
4103,McDonald's,50.949856,6.916848,Burger Joint
4104,Markthalle Körnerstraße,50.948436,6.922124,Restaurant


In [5]:
foursquare_df.to_csv('part2_foursquare_data.csv', index=False)

In [6]:
import time
yelp_api_key = 'BISBc6fQXBhVDrEiGFK6WZFSUJWhaneFpm2naqAHSb2nUJjPFZ1xHpdZfvGCMKbRnIRP-KgjFnqDnkk83PkqEHCuKXfZ66DoCi3F6jBzXWgeikfqam0kchu21LRxZXYx'
headers = {'Authorization': f'Bearer {yelp_api_key}'}

yelp_data = []
for station in df.itertuples():
    params = {
        'latitude': station.latitude,
        'longitude': station.longitude,
        'radius': 1000,
        'categories': 'restaurants,bars',
        'limit': 50
    }
    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)

    if response.status_code == 200:
        data = response.json().get('businesses', [])
        for business in data:
            business_info = {
                'name': business['name'],
                'latitude': business['coordinates']['latitude'],
                'longitude': business['coordinates']['longitude'],
                'rating': business.get('rating', None)
            }
            yelp_data.append(business_info)
    elif response.status_code == 429:
        print("Rate limit reached, request suspended")
        time.sleep(60)  
        continue
    else:
        print(f"Request failed with status code:{response.status_code}, response content:{response.text}")

    time.sleep(1)  

In [7]:
yelp_df = pd.DataFrame(yelp_data)
yelp_df

,name,latitude,longitude,rating
0,Pane e Cioccolata,50.954260,6.924250,4.5
1,Essers Gasthaus,50.956510,6.927020,4.0
2,Shepheard,50.955328,6.932669,4.5
3,Vasco da Gama,50.956440,6.936020,4.5
4,Café Franck,50.956350,6.922420,4.5
...,...,...,...,...
3742,Cafecafe,50.946495,6.924776,4.5
3743,Pöttgen,50.956030,6.918710,4.0
3744,Weinlokal Secco,50.947800,6.922650,4.0
3745,Causas Peru,50.955680,6.905519,4.0


In [8]:
yelp_df.to_csv('part2_yelp_data.csv', index=False)

In [9]:
len(foursquare_df), len(yelp_df)

(4106, 3747)

In [10]:
top_10_restaurants = yelp_df.sort_values(by='rating', ascending=False).head(10)
print(top_10_restaurants)

                               name   latitude  longitude  rating
1873         Restaurant Willomitzer  50.962540   6.997980     5.0
1657                    Kreta Grill  50.971370   7.033960     5.0
2699  Restaurant Gronauer Tannenhof  50.938900   7.064030     5.0
1655                      Neue Post  50.971300   7.049740     5.0
2706                      Helmholtz  50.953490   6.903630     5.0
1644                    Kreta Grill  50.971370   7.033960     5.0
1642                      Neue Post  50.971300   7.049740     5.0
1631               Salva's Pastabar  50.971052   7.073041     5.0
336                     NIMET-GRILL  50.921650   6.959140     5.0
2737                       Woanders  50.948680   6.908037     5.0
